In [1]:
type_weight = {0:0.5, 1:9, 2:0.5}
type_weight_multipliers = type_weight

top = 15

VER = 1
DIR = f'/home/anhphantq/vdb/covisit_ver{VER}'

In [2]:
import os 

try:
    os.mkdir(DIR)
    
    os.mkdir(DIR + '/splitted')
    os.mkdir(DIR + '/lb')
except OSError as error:
    print(error)    

[Errno 17] File exists: '/home/anhphantq/vdb/covisit_ver1'


In [3]:

import pandas as pd

train_df = pd.read_parquet('/home/anhphantq/otto/splitted_data/train.parquet')
test_df = pd.read_parquet('/home/anhphantq/otto/splitted_data/test.parquet')
all_df = pd.concat((train_df, test_df))
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
MAX_TS, MIN_TS

(1661119199, 1659304800)

In [ ]:
import pandas as pd
train_df = pd.read_parquet('/home/anhphantq/otto/data/train.parquet')

#remove 1 week from train df LB to retain distribution 
import numpy as np
min_ts = train_df.ts.values.min()
old_index = np.argwhere(train_df.ts.values - min_ts < 3600 * 24 * 7)
train_df = train_df.drop(old_index.flatten())
test_df = pd.read_parquet('/home/anhphantq/otto/data/test.parquet')
all_df = pd.concat((train_df, test_df))
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
MAX_TS, MIN_TS

Create covisit

In [4]:
import numpy as np
def create_covisit(args): 
    df, result, id = args    
    df = df.sort_values(['session','ts'],ascending=[True,False])
    df['user_num_interact'] = df.groupby('session')['ts'].transform('count')
    df['user_fts'] = df.groupby('session')['ts'].transform('min')
    df['user_lts'] = df.groupby('session')['ts'].transform('max')
    
    df['user_wgt'] = 1 / (1 + np.log((df.user_lts - df.user_fts) + 1)) / \
                    (1 + np.log(df.user_num_interact + 1))
                    
    df['item_wgt'] = df.groupby('aid')['ts'].transform('count')
    df['item_wgt'] = 1 / (1 + np.log(df.item_wgt + 1))
        

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]
    
    


    # ASSIGN WEIGHTS
    
    # df = df[['session', 'aid_x', 'aid_y','type_y', 'user_wgt_y', 'item_wgt_y']].drop_duplicates(['session', 'aid_x', 'aid_y', 'type_y'])
    
    df['wgt'] = df.type_y.map(type_weight) * df.user_wgt_y * df.item_wgt_y 
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    
    
    df = df.groupby(['aid_x','aid_y']).wgt.sum().reset_index().sort_values(['aid_x','wgt'],ascending=[True,False]).reset_index(drop = True)
    df['n'] = df.groupby('aid_x').aid_y.cumcount()
    #SAVE TOP 
    df = df.loc[df.n < top].drop('n', axis = 1)
    result[id] = df
    return id

In [5]:
from tqdm import tqdm
import numpy as np
N = 100
step = all_df.shape[0] // (64 * N)
start = [i * step for i in range(64 * N)]
end = [i + step for i in start]
end[-1] = all_df.shape[0] 

batches = [all_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]
del all_df, train_df, test_df
len(batches)

6400

In [8]:
%%time
from tqdm.notebook import tqdm
import psutil
from multiprocessing import Manager
import multiprocessing
manager = Manager()

result = manager.dict()
#splitted


with tqdm(total=100, desc='CPU%', position=1, leave = False) as cpubar, tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave = False) as rambar:

  with multiprocessing.Pool(64) as pool:
    for rs in tqdm(pool.imap_unordered(create_covisit, zip(batches, [result for _ in range(len(batches))], [i for i in range(len(batches))]), chunksize = 1), total = len(batches)):
      
      rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)

      cpubar.n=psutil.cpu_percent()
      rambar.refresh()
      cpubar.refresh()
      
  
  

CPU%:   0%|          | 0/100 [00:00<?, ?it/s]

RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/6400 [00:00<?, ?it/s]

CPU times: user 51.3 s, sys: 19.4 s, total: 1min 10s
Wall time: 4min 56s


In [12]:
pd.concat(result.values()).to_parquet(DIR + '/splitted/covisit.pqt')

In [ ]:
pd.concat(result.values()).to_parquet(DIR + '/lb/covisit.pqt')